# Aufgabe 5 (Scala: Monte-Carlo)

_Verwenden Sie ausschließlich aus der Lehrveranstaltung bekannte Scala-Konstrukte, dabei sind `var` und seiteneffektbehaftete Methoden/Klassen/Objekte strikt untersagt._

<div class="alert alert-block alert-danger">
<b>WICHTIG: Die folgende Zelle (<tt>import $ivy...</tt>) muss unbedingt <i>vor allen anderen</i> ausgeführt werden!</b>
</div>

In [ ]:
import $ivy.`org.scala-lang.modules::scala-parallel-collections:1.0.4`
import scala.collection.parallel.CollectionConverters._
import scala.collection.parallel.immutable._

Wenn $a$ und $b$ gleichverteilte Zufallsvariablen zwischen $0$ und $1$ sind, liegt der Punkt $(a, b)$ mit einer Wahrscheinlichkeit von $\frac{\pi}{4}$ im Einheitskreis (Kreis um den Ursprung mit Radius $1$). In dieser Aufgabe soll aus einer unendlichen `LazyList` solcher Zahlen mithilfe einer Monte-Carlo-Simulation die Kreiszahl $\pi$ berechnet werden.

In [ ]:
// Unendlich viele Zufallszahlen zwischen 0 und 1
val valuesList : LazyList[Double] = /*...*/
    /*...*/ {def genRandoms: LazyList[Double] = scala.util.Random.nextDouble() #:: genRandoms
    /*...*/ genRandoms}

**a)** Vervollständigen Sie die Funktion `pairing`. Sie erhält eine _unendliche_ Liste von Werten und bildet aus jeweils zwei aufeinander folgenden Werten Paare, die wiederum als _unendliche_ Liste zurückgegeben werden.
```scala
  pairing(LazyList(1, 2, 3, 4, ...)) = LazyList((1,2), (3,4), ...) 
```

In [ ]:
def pairing: LazyList[Double] => LazyList[(Double,Double)] =
   /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
assert(pairing(LazyList(1.0,2.0,3.0,4.0/*,...*/) #::: LazyList.iterate(5.0)(_+1)).take(4) == LazyList((1,2), (3,4)/*, ...*/, (5,6), (7,8))); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Für die sequentielle Verarbeitung von Listen bietet sich oft _Pattern Matching_ und _Rekursion_ an.

Da die übergebene `LazyList` _garantiert unendlich_ ist, genügt für das _Pattern Matching_ ein einziger Fall.

</div>

<details>
<summary><i><span style='background:#b2ebf2'>... Hinweis (Muster links von "=>")</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

**Muster** **links** von `=>` im _Pattern Matching_:
- Zerlege die `LazyList` in ihre zwei Kopfelemente `a` bzw. `b` und die Rest-Liste `rem`.

</div>
<details>
<summary><i><span style='background:#c8e6c9'>... Lösungsvorschlag (Muster links von "=>")</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
case a #:: b #:: rem => // ... 
```

</div>
</details>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>... Hinweis (Ergebnis rechts von "=>")</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

**Ergebnis** **rechts** von `=>` im _Pattern Matching_:
- Baue aus den beiden Kopfelementen ein Tupel `(a,b)` und mache es zum Kopfelement der Liste, die aus dem rekursiven Aufruf von `pairing` mit der Restliste `rem` entsteht.

</div>
<details>
<summary><i><span style='background:#c8e6c9'>... Lösungsvorschlag (Ergebnis rechts von "=>")</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
/*...*/ => (a,b) #:: pairing(rem) 
```

</div>
</details>
</details>

</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def pairing: LazyList[Double] => LazyList[(Double,Double)] = {
   case a #:: b #:: rem => (a,b) #:: pairing(rem)
} 
```

</div>
</details>

**b)** Vervollständigen Sie nun die Funktion `testN`. Sie soll die ersten `n` Zahlenpaare der übergebenen `LazyList` <ins>**parallel**</ins> dahingehend überprüfen, ob diese im Einheitskreis liegen. Das Ergebnis dieser Berechnung soll als `List` der Länge `n` zurückgegeben werden.
```scala
  testN(LazyList((0.1,0.2), (0.8,0.9), ...))(2) = List(true, false) 
```

In [ ]:
def testN: LazyList[(Double,Double)] => Int => List[Boolean] =
   pairs => n =>
      /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
assert(testN((0.1,0.2) #:: (0.8,0.9) #:: valuesList.zip(valuesList))(2) == List(true, false)); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Aus der übergebenen (_garantiert unendlichen_) `LazyList` `pairs` benötigen wir laut Aufgabenstellung nur _"die ersten `n` Zahlenpaare"_. Dafür eignet sich die Listenfunktion `take`.

Außerdem soll die Überprüfung **<ins>parallel</ins>** erfolgen, d.h. wir müssen den extrahierten Teil der `LazyList` in eine parallele Datenstruktur umwandeln, was hier mit `.par` möglich ist.
</div>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag (Anfang)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
    pairs.take(n).par/*...*/ 
```

</div>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>... Hinweis (Ende)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Das Zwischenergebnis nach `take` und `par` ist ein (endlicher) `ParVector` von Tupeln der Form `(x,y)` - wobei `x` und `y` vom Typ `Double` sind:
- `ParVector((a,b), (c,d), (e,f), ...)`

Für jedes Tupel `(x,y)` soll nun geprüft werden, ob die davon dargestellten Koordinaten im Einheitskreis liegen, d.h. ob $x^2 + y^2 < 1$ gilt. Dazu eignet sich die Verwendung der `map`-Funktion des `ParVector` mit einem passenden _Pattern Matching_ als Abbildungsfunktion vom Typ `(Double,Double) => Boolean`.

Als Zwischenergebnis ergibt sich daraus nunmehr ein `ParVector[Boolean]`. Die vorgegebene Signatur der Funktion erfordert aber eine `List[Boolean]`. Daher müssen wir den `ParVector` abschließend noch mittels `.toList` in eine klassische Scala-`List` "umwandeln".

</div>

</details>
<details>
<summary><i><span style='background:#c8e6c9'>... Lösungsvorschlag (Ende)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
    /*...*/.map(_ match {
         case (a,b) => (a*a) + (b*b) < 1
      }).toList 
```

</div>
</details>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def testN: LazyList[(Double,Double)] => Int => List[Boolean] =
   pairs => n =>
      pairs.take(n).par.map(_ match {
         case (a,b) => (a*a) + (b*b) < 1
      }).toList 
```

</div>
</details>

**c)** Vervollständigen Sie die Funktion `pi`. Sie wendet mithilfe der Werte `valuesList` eine Monte-Carlo-Simulation mit `n` Punkten an, um $\pi$ zu berechnen und zurückzugeben.
```scala
  pi(10000000) = 3.1416 
```

In [ ]:
def pi: Int => Double = n =>
   /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
assert(math.abs(math.Pi-pi(1000000)) < 0.07); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Zur Erinnerung (_Monte-Carlo_-Simulation für $\pi$):
- "Wenn $a$ und $b$ gleichverteilte Zufallsvariablen zwischen $0$ und $1$ sind, liegt der Punkt $(a, b)$ mit einer Wahrscheinlichkeit von $\frac{\pi}{4}$ im Einheitskreis."
- Der Bruchteil $\frac{1}{4}$ kommt daher, dass $(a, b)$ wegen $0 \leq a,b \leq 1$ _nur im ersten_ (von _vier_) Einheitsquadranten liegen kann.

Wenn wir also $n$ derartige _"zufällige Punkte"_ untersuchen (und $n$ groß genug ist), dann sollten ca. $N := n \cdot \frac{\pi}{4}$ viele davon in demjenigen Teil des Einheitskreises sein, der im ersten Einheitsquadranten liegt.
Um den Wert von $\pi$ zu bestimmen, lösen wir die Gleichung entsprechend auf: $\pi = 4 \cdot \frac{N}{n}$

Wir müssen also in der Funktion `pi` eine _Monte-Carlo-Simulation_ mit `n` Punkten $(a,b)$ durchführen und die Anzahl $N$ derjenigen bestimmen, die im Einheitskreis liegen. Dabei müssen die Punkte im (ersten) Einheitsquadranten liegen: $0 \leq a,b \leq 1$.

</div>

<details>
<summary><i><span style='background:#b2ebf2'>... Fortsetzung (innen)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Glücklicherweise stellt uns die vorgegebene Konstante `valuesList` _"unendlich viele Zufallszahlen zwischen 0 und 1"_ bereit.

Die Zufallszahlen aus `valuesList` müssen noch zu Paaren zusammengesetzt werden, damit sie Punkte bzw. Koordinaten im Einheitsquadranten werden. Das erledigt die Funktion `pairing` aus Teilaufgabe **a)**.

Auf das Zwischenergebnis von `pairing` wenden wir `testN` aus Teilaufgabe **b)** an. Damit erhalten wir für die ersten $n$ Punkte jeweils eine Wahrheitsaussage (`true`/`false`), je nachdem ob der Punkt im Einheitskreis ist - oder eben nicht.

</div>

<details>
<summary><i><span style='background:#c8e6c9'>... Lösungsvorschlag (innen)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
/*...*/testN(pairing(valuesList))(n)/*...*/
```

</div>
</details>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>... Fortsetzung (außen)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Um $N$ zu bestimmen, müssen wir zählen (lassen), wie oft `true` im vorangehenden Zwischenergebnis (siehe Hinweis unter _"... Fortsetzung (innen)"_) vorkommt, also wie viele Punkte im Einheitskreis lagen.
- Das erledigt die API-Funktion höherer Ordnung von List: `count(p: A => Boolean): Int`
- Als Prädikat genügt dabei die Abbildung `(x => x)`, denn wir haben ja bereits Wahrheitswerte vorliegen.

Abschließend müssen wir für $\pi$ nur noch obige Formel anwenden: $\pi = 4 \cdot \frac{N}{n}$

</div>

<details>
<summary><i><span style='background:#c8e6c9'>... Lösungsvorschlag (außen)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
4 * (/*...*/).count(x => x) / n.toDouble
```

</div>
</details>
</details>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def pi: Int => Double = n =>
   4 * (testN(pairing(valuesList))(n)).count(x => x) / n.toDouble 
```

</div>
</details>